In [1]:
import pandas as pd
import numpy as np
import glob

all_files = glob.glob("Data/*.csv")
df_list = ['ADB', 'ADS', 'BPB', 'BPS', 'CDB', 'CDS', 'ECB', 'ECS', 'JYB', 'JYS', 'SFB', 'SFS']
name_list = ['ADB', 'ADS', 'BPB', 'BPS', 'CDB', 'CDS', 'ECB', 'ECS', 'JYB', 'JYS', 'SFB', 'SFS']

#read into corresponding dfs 
for i, csv in enumerate(all_files):
    df_list[i] = pd.read_csv(csv)
    name = name_list[i]
    df_list[i].columns = [name+'Date',name+'Open',name+'High',name+'Low',name+'Close',name+'Volume',name+'Open Interest']
    df_list[i].drop(columns=[name+'Open',name+'High',name+'Low',], inplace=True)
    df_list[i].drop(columns=[name+'Volume',name+'Open Interest'], inplace=True)
    df_list[i].set_index(name+'Date', inplace=True)
    df_list[i].rename_axis("Date", inplace=True)

specs = pd.read_csv("contract_specs.csv")
specs.set_index("Ticker", inplace=True)

In [2]:
specs.head()

,Multiplier,Tick Size,Exchange Fees,Commission,NFA Fees,Slippage,Sector
Ticker,,,,,,,
ES,50,0.25000,1.18,0.85,0.02,2,IND
CL,1000,0.01000,1.50,0.85,0.02,2,ENG
TF,50,0.10000,1.18,0.85,0.02,2,IND
VX,1000,0.05000,1.44,0.85,0.02,2,NaN
EC,125000,0.00005,1.60,0.85,0.02,2,CUR


In [3]:
#merge dfs
total = df_list[0].copy()
for i in range(1, len(df_list)):
    total = pd.merge(total, df_list[i], on='Date', how='outer', sort=True).fillna(method='ffill')
# total.sort_index(inplace=True)

In [4]:
total

,ADBClose,ADSClose,BPBClose,BPSClose,CDBClose,CDSClose,ECBClose,ECSClose,JYBClose,JYSClose,SFBClose,SFSClose
Date,,,,,,,,,,,,
19791210,NaN,NaN,1.3831,2.1555,0.66355,0.86050,NaN,NaN,1.27050,0.43360,1.2971,0.6355
19791211,NaN,NaN,1.4141,2.1865,0.66635,0.86330,NaN,NaN,1.26770,0.43080,1.3022,0.6406
19791212,NaN,NaN,1.4121,2.1845,0.66455,0.86150,NaN,NaN,1.26440,0.42750,1.3009,0.6393
19791213,NaN,NaN,1.4171,2.1895,0.66315,0.86010,NaN,NaN,1.25860,0.42170,1.3003,0.6387
19791214,NaN,NaN,1.4316,2.2040,0.65795,0.85490,NaN,NaN,1.25280,0.41590,1.2974,0.6358
...,...,...,...,...,...,...,...,...,...,...,...,...
20220214,0.71225,0.71225,1.3518,1.3518,0.78520,0.78520,1.13025,1.13025,0.86485,0.86485,1.0811,1.0811
20220215,0.71530,0.71530,1.3532,1.3532,0.78530,0.78530,1.13660,1.13660,0.86500,0.86500,1.0811,1.0811
20220216,0.72030,0.72030,1.3597,1.3597,0.78875,0.78875,1.13960,1.13960,0.86670,0.86670,1.0865,1.0865


In [57]:
def test(df, num_stdevs=0.5, min_days_held=5):
    name_list = ['ADB', 'ADS', 'BPB', 'BPS', 'CDB', 'CDS', 'ECB', 'ECS', 'JYB', 'JYS', 'SFB', 'SFS']
    real_list = ['AD', 'BP', 'CD', 'EC', 'JY', 'SF']
    lookback = 252
#     start = 4 - lookback
    start = df.index.get_loc(20070102) - lookback
    
    for i in range(0, len(name_list), 2):
        real_name = real_list[int(i/2)]
        name_1 = name_list[i]
        name_2 = name_list[i+1]
        
        df[real_name+' Returns'] = (df[name_1+'Close'] - (df[name_1+'Close'].shift(1))) / (df[name_2+'Close'].shift(1))

        df[real_name+' Vol'] = df[real_name+' Returns'].rolling(lookback).std()
        df[real_name+' Pct Change'] = (df[name_1+'Close'] - (df[name_1+'Close'].shift(lookback))) / (df[name_2+'Close'].shift(lookback))
        df[real_name+' Trend'] = df[real_name+' Pct Change'] / df[real_name+' Vol']
        
        df[real_name+' P&L'] = 0
        df[real_name+' Position'] = 0
    
    df['M with biggest'] = df.loc[:,['AD Trend', 'BP Trend', 'CD Trend', 'EC Trend', 'JY Trend', 'SF Trend']].abs().idxmax(axis=1)

    df['Days since last trade'] = 0
    df['Total P&L'] = 0
#     print(df['Total P&L'])
    
    # Initial Position 
    init_market_trend = df.loc[df.index[start + lookback], 'M with biggest']
    print(init_market_trend)
    init_market = init_market_trend[:2]
    
    init_sum = 0
    for market in real_list:
        print(np.nan_to_num(df.loc[df.index[start + lookback],market+' Vol']))
        print(specs.loc[market, 'Multiplier'])
        init_sum += 1 / (np.nan_to_num(df.loc[df.index[start + lookback],market+' Vol']) * specs.loc[market, 'Multiplier'])
    print("Init sum " + str(init_sum))
    
    init_sign = np.sign(df.loc[df.index[start + lookback], init_market+' Trend'])
    
    df.loc[df.index[start + lookback], init_market+' Position'] = init_sign * 10 * (1/(df.loc[df.index[start + lookback], init_market+' Vol'] * specs.loc[init_market, "Multiplier"])) / init_sum
    print(1/df.loc[20070102, 'BP Vol'])
    print((1/(df.loc[df.index[start + lookback], init_market+' Vol'])))
    
    df['Current Market'] = np.nan
    df.loc[df.index[start + lookback], 'Current Market'] = init_market 
    df.loc[df.index[start + lookback], init_market+' P&L'] = df.loc[df.index[start + lookback+1-1], init_market+' Position'] * specs.loc[init_market, "Multiplier"] * (df.loc[df.index[start+lookback+1], init_market+'BClose'] - df.loc[df.index[start+lookback], init_market+'BClose'])
    print((df.loc[df.index[start+lookback+1], init_market+'BClose'] - df.loc[df.index[start+lookback], init_market+'BClose']))
    print("CLEAR")
    
    # Initial transaction cost
    tick_size_init= specs.loc[init_market, "Tick Size"]
    mult_init = specs.loc[init_market, "Multiplier"]
    exch_init = specs.loc[init_market, "Exchange Fees"]
    comm_init = specs.loc[init_market, "Commission"]
    nfa_init = specs.loc[init_market, "NFA Fees"]
    init_pos = df.loc[df.index[start + lookback+1-1], init_market+' Position']
    
    df.loc[df.index[start+lookback], 'Total P&L'] = - np.abs(init_pos) * (tick_size_init * mult_init + exch_init + comm_init + nfa_init)
    
    for i,date in enumerate(df.index[start + lookback + 1:-1]):
        yesterday = start + lookback + i + 1 - 1 
        current_date = start + lookback + i + 1
#         if i < 5:
#             print(date)
#             print(df.index[yesterday])
#             print(i)
        no_trade = False 
        
        # Current Position
        new_biggest_market = np.NaN
        if not (pd.isnull(df.loc[date, 'M with biggest'])):
            new_biggest_market = (df.loc[date, 'M with biggest'])[:2]
        if new_biggest_market == df.loc[df.index[yesterday], 'Current Market']:
#             print("No trade ")
#             print(new_biggest_market)
#             print(df.loc[df.index[start+lookback+i-1], 'Current Market'])
            no_trade = True
        
        # Min days passed
        if df.loc[df.index[yesterday], 'Days since last trade'] < min_days_held:
            no_trade = True
        
        # Pullback
        if no_trade == False:
            curr_return = df.loc[date, new_biggest_market+' Returns']
            trend = df.loc[date, new_biggest_market+' Trend']
            #check sign
            if np.sign(curr_return) == np.sign(trend):
                #check magn
                if (curr_return >= num_stdevs * (df.loc[date, new_biggest_market+' Vol'])):
                    pass
                else:
                    no_trade = True
            else:
                no_trade = True
        
        if no_trade:
            new_biggest_market = df.loc[df.index[yesterday], 'Current Market']
            if (pd.isnull(new_biggest_market)):
                continue
            df.loc[date, 'Current Market'] = new_biggest_market
            df.loc[date, new_biggest_market+' Position'] = df.loc[df.index[yesterday], new_biggest_market+' Position']
            df.loc[date, 'Days since last trade'] = df.loc[df.index[yesterday], 'Days since last trade'] + 1
            
            pos_size = df.loc[date, new_biggest_market+' Position']
            mult = specs.loc[new_biggest_market, "Multiplier"]
            pandL = df.loc[df.index[current_date + 1], new_biggest_market+'BClose'] - df.loc[date, new_biggest_market+'BClose']
            df.loc[date, new_biggest_market+' P&L'] += pos_size * mult * pandL
#             print("Position " + str(date) + " "+ str(pos_size * mult * pandL))
        else:
            print("Making trade " + str(date))
            trend_i = df.loc[date, new_biggest_market+' Trend']
            vol_i = df.loc[date, new_biggest_market+' Vol']
            mult_i = specs.loc[new_biggest_market, "Multiplier"]
            sum_j = 0
            for market in real_list:
                sum_j += 1 / (np.nan_to_num(df.loc[date,market+' Vol']) * specs.loc[market, 'Multiplier'])
            pos_sign = np.sign(trend_i)
            position_size = pos_sign * 10 * (1/(vol_i * mult_i)) / sum_j
            
            df.loc[date, new_biggest_market+' Position'] = position_size 
            df.loc[date, 'Current Market'] = new_biggest_market
            df.loc[date, 'Days since last trade'] = 0
            
            # account for last day of price change of old position
#             old_market = df.loc[df.index[start + i+lookback+1-1], 'Current Market']
#             old_pos_size = df.loc[df.index[start+i+lookback+1-1], old_market+' Position']
#             mult = specs.loc[old_market, "Multiplier"]
#             df.loc[date, old_market+' P&L'] += np.abs(old_pos_size) * mult * (df.loc[df.index[start + i+lookback+1-1], old_market+'BClose'] - df.loc[date, old_market+'BClose'])
            
            
            #transaction costs
            # sell old position
#             old_market = df.loc[df.index[yesterday], 'Current Market']
#             tick_size0 = specs.loc[old_market, "Tick Size"]
#             mult0 = specs.loc[old_market, "Multiplier"]
#             exch0 = specs.loc[old_market, "Exchange Fees"]
#             comm0 = specs.loc[old_market, "Commission"]
#             nfa0 = specs.loc[old_market, "NFA Fees"]
#             old_pos = df.loc[df.index[yesterday], old_market+' Position']
            
#             df.loc[date,'Total P&L'] -= np.abs(old_pos) * (tick_size0 * mult0 + exch0 + comm0 + nfa0)
            
            # buy into new position 
            tick_size1 = specs.loc[new_biggest_market, "Tick Size"]
            exch1 = specs.loc[new_biggest_market, "Exchange Fees"]
            comm1 = specs.loc[new_biggest_market, "Commission"]
            nfa1 = specs.loc[new_biggest_market, "NFA Fees"]
            
            df.loc[date, 'Total P&L'] -= 2 * np.abs(position_size) * (tick_size1 * mult_i + exch1 + comm1 + nfa1)
#     print(df["BP Position"])
#     print(df)
    new_list = ["AD", 'BP', 'CD', 'EC', 'JY', 'SF']
    for market in new_list:
        df['Total P&L'] += df[market+" P&L"].cumsum()
#         print(df[market+" P&L"].cumsum())
#     print(df['Total P&L'])
# #     df = df.reindex(sorted(df.columns), axis=1)

#     for i,date in enumerate(df.index):
#         if i < 10:
#             for market in real_list:
#                 if df.loc[date, market+" P&L"] != 0:
#                     print("Not 0! " + str(df.loc[date,market+" P&L"]))

    return df

In [58]:
looped = test(total)
looped.tail()

BP Trend
0.0055427758443840425
100000
0.004909341898794992
62500
0.004543670686106805
100000
0.004686978430556544
125000
0.005163964414945516
125000
0.00545489885619646
125000
Init sum 0.011986732115027151
203.69328936846955
203.69328936846955
-0.02310000000000012
CLEAR
Making trade 20070402
Making trade 20070410
Making trade 20070420
Making trade 20070820
Making trade 20070912
Making trade 20070925
Making trade 20071012
Making trade 20071023
Making trade 20071031
Making trade 20071114
Making trade 20071217
Making trade 20080110
Making trade 20080208
Making trade 20080219
Making trade 20080922
Making trade 20081216
Making trade 20091006
Making trade 20100803
Making trade 20100817
Making trade 20101014
Making trade 20101102
Making trade 20101231
Making trade 20110118
Making trade 20110222
Making trade 20110630
Making trade 20110712
Making trade 20110829
Making trade 20110919
Making trade 20111014
Making trade 20111103
Making trade 20111111
Making trade 20120125
Making trade 20121004
Mak

,ADBClose,ADSClose,BPBClose,BPSClose,CDBClose,CDSClose,ECBClose,ECSClose,JYBClose,JYSClose,SFBClose,SFSClose,AD Returns,AD Vol,AD Pct Change,AD Trend,AD P&L,AD Position,BP Returns,BP Vol,BP Pct Change,BP Trend,BP P&L,BP Position,CD Returns,CD Vol,CD Pct Change,CD Trend,CD P&L,CD Position,EC Returns,EC Vol,EC Pct Change,EC Trend,EC P&L,EC Position,JY Returns,JY Vol,JY Pct Change,JY Trend,JY P&L,JY Position,SF Returns,SF Vol,SF Pct Change,SF Trend,SF P&L,SF Position,M with biggest,Days since last trade,Total P&L,Current Market
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20220214,0.71225,0.71225,1.3518,1.3518,0.78520,0.78520,1.13025,1.13025,0.86485,0.86485,1.0811,1.0811,-0.000561,0.005892,-0.084316,-14.311086,0.0,0.0,-0.002141,0.004142,-0.029253,-7.063015,213.756996,2.442937,0.000510,0.004380,-0.004754,-1.085472,0.0,0.0,-0.003570,0.003808,-0.075276,-19.770254,0.0,0.0,-0.003572,0.003645,-0.087665,-24.051897,0.0,0.0,-0.000462,0.004302,-0.046074,-10.710905,0.0,0.0,JY Trend,109,-30041.012425,BP
20220215,0.71530,0.71530,1.3532,1.3532,0.78530,0.78530,1.13660,1.13660,0.86500,0.86500,1.0811,1.0811,0.004282,0.005898,-0.078903,-13.377453,0.0,0.0,0.001036,0.004136,-0.024672,-5.964770,992.443197,2.442937,0.000127,0.004378,-0.002984,-0.681542,0.0,0.0,0.005618,0.003805,-0.063938,-16.801904,0.0,0.0,0.000173,0.003645,-0.087362,-23.967691,0.0,0.0,0.000000,0.004272,-0.038189,-8.940062,0.0,0.0,JY Trend,110,-29048.569228,BP
20220216,0.72030,0.72030,1.3597,1.3597,0.78875,0.78875,1.13960,1.13960,0.86670,0.86670,1.0865,1.0865,0.006990,0.005914,-0.074660,-12.624978,0.0,0.0,0.004803,0.004114,-0.027977,-6.799648,351.172208,2.442937,0.004393,0.004386,-0.000063,-0.014454,0.0,0.0,0.002639,0.003800,-0.065211,-17.160109,0.0,0.0,0.001965,0.003645,-0.087398,-23.978339,0.0,0.0,0.004995,0.004279,-0.036194,-8.457761,0.0,0.0,JY Trend,111,-28697.397020,BP
20220217,0.71935,0.71935,1.3620,1.3620,0.78745,0.78745,1.13690,1.13690,0.87065,0.87065,1.0875,1.0875,-0.001319,0.005858,-0.087338,-14.909149,0.0,0.0,0.001692,0.004112,-0.029040,-7.062487,-290.098781,2.442937,-0.001648,0.004377,-0.006373,-1.455956,0.0,0.0,-0.002369,0.003799,-0.069752,-18.363039,0.0,0.0,0.004558,0.003655,-0.085012,-23.258480,0.0,0.0,0.000920,0.004280,-0.034953,-8.166654,0.0,0.0,JY Trend,112,-28987.495800,BP
20220218,0.71865,0.71865,1.3601,1.3601,0.78465,0.78465,1.13375,1.13375,0.86910,0.86910,1.0864,1.0864,-0.000973,0.005837,-0.094959,-16.267246,0.0,0.0,-0.001395,0.004102,-0.034735,-8.467075,0.000000,0.000000,-0.003556,0.004382,-0.011591,-2.645322,0.0,0.0,-0.002771,0.003791,-0.076275,-20.120624,0.0,0.0,-0.001780,0.003643,-0.090723,-24.900504,0.0,0.0,-0.001011,0.004279,-0.037233,-8.700753,0.0,0.0,JY Trend,0,-28987.495800,NaN


In [59]:
looped.iloc[6813:6830,:]
# looped["JY Position"].head(10)

,ADBClose,ADSClose,BPBClose,BPSClose,CDBClose,CDSClose,ECBClose,ECSClose,JYBClose,JYSClose,SFBClose,SFSClose,AD Returns,AD Vol,AD Pct Change,AD Trend,AD P&L,AD Position,BP Returns,BP Vol,BP Pct Change,BP Trend,BP P&L,BP Position,CD Returns,CD Vol,CD Pct Change,CD Trend,CD P&L,CD Position,EC Returns,EC Vol,EC Pct Change,EC Trend,EC P&L,EC Position,JY Returns,JY Vol,JY Pct Change,JY Trend,JY P&L,JY Position,SF Returns,SF Vol,SF Pct Change,SF Trend,SF P&L,SF Position,M with biggest,Days since last trade,Total P&L,Current Market
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20061226,0.52725,0.7805,1.9457,1.9545,0.84455,0.8632,1.4376,1.3149,1.0241,0.8486,1.0346,0.8238,-0.001280,0.005542,0.077735,14.026097,0.0,0.0,-0.001277,0.004916,0.120030,24.416977,0.000000,0.000000,-0.002773,0.004581,-0.007902,-1.725192,0.0,0.0,-0.001443,0.004667,0.079883,17.117859,0.0,0.0,-0.002117,0.005204,-0.070299,-13.508775,0.0,0.0,-0.000728,0.005442,0.033312,6.120864,0.0,0.0,BP Trend,0,0.000000,NaN
20061227,0.52935,0.7826,1.9481,1.9569,0.84485,0.8635,1.4398,1.3171,1.0270,0.8515,1.0326,0.8218,0.002691,0.005538,0.084928,15.336752,0.0,0.0,0.001228,0.004911,0.124841,25.420479,0.000000,0.000000,0.000348,0.004580,-0.006515,-1.422359,0.0,0.0,0.001673,0.004666,0.083543,17.905806,0.0,0.0,0.003417,0.005205,-0.069636,-13.377856,0.0,0.0,-0.002428,0.005443,0.032725,6.012155,0.0,0.0,BP Trend,0,0.000000,NaN
20061228,0.53425,0.7875,1.9545,1.9633,0.84495,0.8636,1.4420,1.3193,1.0250,0.8495,1.0348,0.8240,0.006261,0.005535,0.098324,17.763436,0.0,0.0,0.003270,0.004908,0.132522,27.003216,0.000000,0.000000,0.000116,0.004558,0.000703,0.154224,0.0,0.0,0.001670,0.004663,0.087948,18.859280,0.0,0.0,-0.002349,0.005167,-0.062115,-12.021723,0.0,0.0,0.002677,0.005442,0.038844,7.138431,0.0,0.0,BP Trend,0,0.000000,NaN
20061229,0.53335,0.7866,1.9484,1.9572,0.84155,0.8602,1.4463,1.3236,1.0239,0.8484,1.0362,0.8254,-0.001143,0.005519,0.089234,16.167867,0.0,0.0,-0.003107,0.004891,0.136825,27.976634,0.000000,0.000000,-0.003937,0.004544,-0.010123,-2.227833,0.0,0.0,0.003259,0.004667,0.092119,19.739587,0.0,0.0,-0.001295,0.005162,-0.059785,-11.581150,0.0,0.0,0.001699,0.005442,0.040403,7.423744,0.0,0.0,BP Trend,0,0.000000,NaN
20070102,0.54135,0.7946,1.9654,1.9742,0.84085,0.8595,1.4560,1.3333,1.0255,0.8500,1.0414,0.8306,0.010170,0.005543,0.093767,16.916990,0.0,0.0,0.008686,0.004909,0.140951,28.710671,-3925.436006,2.718917,-0.000814,0.004544,-0.012087,-2.660272,0.0,0.0,0.007328,0.004687,0.098898,21.100657,0.0,0.0,0.001886,0.005164,-0.057477,-11.130331,0.0,0.0,0.006300,0.005455,0.045152,8.277336,0.0,0.0,BP Trend,0,-3949.144960,BP
20070103,0.53745,0.7907,1.9423,1.9511,0.83655,0.8552,1.4438,1.3211,1.0218,0.8463,1.0322,0.8214,-0.004908,0.005552,0.087099,15.686525,0.0,0.0,-0.011701,0.004965,0.130564,26.294361,-1070.573456,2.718917,-0.005003,0.004553,-0.018910,-4.153366,0.0,0.0,-0.009150,0.004725,0.089646,18.974396,0.0,0.0,-0.004353,0.005170,-0.060812,-11.761753,0.0,0.0,-0.011076,0.005500,0.034091,6.197836,0.0,0.0,BP Trend,1,-4996.009463,BP
20070104,0.52975,0.7830,1.9360,1.9448,0.83245,0.8511,1.4358,1.3131,1.0234,0.8479,1.0272,0.8164,-0.009738,0.005562,0.066973,12.042222,0.0,0.0,-0.003229,0.004883,0.109978,22.524519,-2379.052125,2.718917,-0.004794,0.004549,-0.029072,-6.391602,0.0,0.0,-0.006056,0.004640,0.066131,14.250898,0.0,0.0,0.001891,0.005067,-0.073887,-14.580477,0.0,0.0,-0.006087,0.005375,0.007815,1.453799,0.0,0.0,BP Trend,2,-7375.061588,BP
20070105,0.52385,0.7771,1.9220,1.9308,0.83565,0.8543,1.4279,1.3052,1.0266,0.8511,1.0253,0.8145,-0.007535,0.005551,0.048774,8.786764,0.0,0.0,-0.007199,0.004886,0.093805,19.199528,1257.498980,2.718917,0.003760,0.004527,-0.032967,-7.281793,0.0,0.0,-0.006016,0.004627,0.050530,10.920608,0.0,0.0,0.003774,0.005074,-0.070632,-13.921033,0.0,0.0,-0.002327,0.005332,-0.005575,-1.045570,0.0,0.0,BP Trend,3,-6117.562607,BP
20070108,0.52505,0.7783,1.9294,1.9382,0.83385,0.8525,1.4288,1.3061,1.0256,0.8501,1.0246,0.8138,0.001544,0.005551,0.049819,8.974095,0.0,0.0,0.003833,0.

In [16]:
looped.tail()

,ADBClose,ADSClose,BPBClose,BPSClose,CDBClose,CDSClose,ECBClose,ECSClose,JYBClose,JYSClose,SFBClose,SFSClose,AD Returns,AD Vol,AD Pct Change,AD Trend,AD P&L,AD Position,BP Returns,BP Vol,BP Pct Change,BP Trend,BP P&L,BP Position,CD Returns,CD Vol,CD Pct Change,CD Trend,CD P&L,CD Position,EC Returns,EC Vol,EC Pct Change,EC Trend,EC P&L,EC Position,JY Returns,JY Vol,JY Pct Change,JY Trend,JY P&L,JY Position,SF Returns,SF Vol,SF Pct Change,SF Trend,SF P&L,SF Position,M with biggest,Days since last trade,Total P&L,Current Market
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20220214,0.71225,0.71225,1.3518,1.3518,0.78520,0.78520,1.13025,1.13025,0.86485,0.86485,1.0811,1.0811,-0.000561,0.005892,-0.084316,-14.311086,0.0,0.0,-0.002141,0.004142,-0.029253,-7.063015,213.756996,2.442937,0.000510,0.004380,-0.004754,-1.085472,0.0,0.0,-0.003570,0.003808,-0.075276,-19.770254,0.0,0.0,-0.003572,0.003645,-0.087665,-24.051897,0.0,0.0,-0.000462,0.004302,-0.046074,-10.710905,0.0,0.0,JY Trend,109,-30105.594637,BP
20220215,0.71530,0.71530,1.3532,1.3532,0.78530,0.78530,1.13660,1.13660,0.86500,0.86500,1.0811,1.0811,0.004282,0.005898,-0.078903,-13.377453,0.0,0.0,0.001036,0.004136,-0.024672,-5.964770,992.443197,2.442937,0.000127,0.004378,-0.002984,-0.681542,0.0,0.0,0.005618,0.003805,-0.063938,-16.801904,0.0,0.0,0.000173,0.003645,-0.087362,-23.967691,0.0,0.0,0.000000,0.004272,-0.038189,-8.940062,0.0,0.0,JY Trend,110,-29113.151440,BP
20220216,0.72030,0.72030,1.3597,1.3597,0.78875,0.78875,1.13960,1.13960,0.86670,0.86670,1.0865,1.0865,0.006990,0.005914,-0.074660,-12.624978,0.0,0.0,0.004803,0.004114,-0.027977,-6.799648,351.172208,2.442937,0.004393,0.004386,-0.000063,-0.014454,0.0,0.0,0.002639,0.003800,-0.065211,-17.160109,0.0,0.0,0.001965,0.003645,-0.087398,-23.978339,0.0,0.0,0.004995,0.004279,-0.036194,-8.457761,0.0,0.0,JY Trend,111,-28761.979231,BP
20220217,0.71935,0.71935,1.3620,1.3620,0.78745,0.78745,1.13690,1.13690,0.87065,0.87065,1.0875,1.0875,-0.001319,0.005858,-0.087338,-14.909149,0.0,0.0,0.001692,0.004112,-0.029040,-7.062487,-290.098781,2.442937,-0.001648,0.004377,-0.006373,-1.455956,0.0,0.0,-0.002369,0.003799,-0.069752,-18.363039,0.0,0.0,0.004558,0.003655,-0.085012,-23.258480,0.0,0.0,0.000920,0.004280,-0.034953,-8.166654,0.0,0.0,JY Trend,112,-29052.078012,BP
20220218,0.71865,0.71865,1.3601,1.3601,0.78465,0.78465,1.13375,1.13375,0.86910,0.86910,1.0864,1.0864,-0.000973,0.005837,-0.094959,-16.267246,0.0,0.0,-0.001395,0.004102,-0.034735,-8.467075,0.000000,0.000000,-0.003556,0.004382,-0.011591,-2.645322,0.0,0.0,-0.002771,0.003791,-0.076275,-20.120624,0.0,0.0,-0.001780,0.003643,-0.090723,-24.900504,0.0,0.0,-0.001011,0.004279,-0.037233,-8.700753,0.0,0.0,JY Trend,0,-29052.078012,NaN


In [44]:
looped.loc[total.index[6813:6840], ["BP Position", "AD P&L", "EC P&L", "BP P&L", "JY P&L", "CD P&L", "SF P&L", "Total P&L"]]

NameError: name 'looped' is not defined

In [45]:
for i,date in enumerate(total.index):
    if date == 20070102:
        print(i)


6817


In [46]:
looped.iloc[6870:6890,:]

NameError: name 'looped' is not defined

In [47]:
total

,ADBClose,ADSClose,BPBClose,BPSClose,CDBClose,CDSClose,ECBClose,ECSClose,JYBClose,JYSClose,...,SF Returns,SF Vol,SF Pct Change,SF Trend,SF P&L,SF Position,M with biggest,Days since last trade,Total P&L,Current Market
Date,,,,,,,,,,,,,,,,,,,,,
19791210,NaN,NaN,1.3831,2.1555,0.66355,0.86050,NaN,NaN,1.27050,0.43360,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,0.000000,NaN
19791211,NaN,NaN,1.4141,2.1865,0.66635,0.86330,NaN,NaN,1.26770,0.43080,...,0.008025,NaN,NaN,NaN,0.0,0.0,NaN,0,0.000000,NaN
19791212,NaN,NaN,1.4121,2.1845,0.66455,0.86150,NaN,NaN,1.26440,0.42750,...,-0.002029,NaN,NaN,NaN,0.0,0.0,NaN,0,0.000000,NaN
19791213,NaN,NaN,1.4171,2.1895,0.66315,0.86010,NaN,NaN,1.25860,0.42170,...,-0.000939,NaN,NaN,NaN,0.0,0.0,NaN,0,0.000000,NaN
19791214,NaN,NaN,1.4316,2.2040,0.65795,0.85490,NaN,NaN,1.25280,0.41590,...,-0.004540,NaN,NaN,NaN,0.0,0.0,NaN,0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220214,0.71225,0.71225,1.3518,1.3518,0.78520,0.78520,1.13025,1.13025,0.86485,0.86485,...,-0.000462,0.004302,-0.046074,-10.710905,0.0,0.0,JY Trend,109,6365.640696,BP
20220215,0.71530,0.71530,1.3532,1.3532,0.78530,0.78530,1.13660,1.13660,0.86500,0.86500,...,0.000000,0.004272,-0.038189,-8.940062,0.0,0.0,JY Trend,110,6365.640696,BP
20220216,0.72030,0.72030,1.3597,1.3597,0.78875,0.78875,1.13960,1.13960,0.86670,0.86670,...,0.004995,0.004279,-0.036194,-8.457761,0.0,0.0,JY Trend,111,6365.640696,BP


In [4]:
def trade_logic(df_raw, market_list= ['AD', 'BP', 'CD', 'EC', 'JY', 'SF']):
    #df, specs, look_back, num_stdevs, min_days_held, market_list
    name_list = ['ADB', 'ADS', 'BPB', 'BPS', 'CDB', 'CDS', 'ECB', 'ECS', 'JYB', 'JYS', 'SFB', 'SFS']
    real_list = ['AD', 'BP', 'CD', 'EC', 'JY', 'SF']
    
    lookback = 252
    num_stdevs = 0.5
    start = 20070102
    
    df_start = df_raw.index.get_loc(start)
    i_start = df_start - lookback 
    df = df_raw.iloc[i_start:, :].copy()
    
    transac_cost = {}
    
    df['Sizing Sum'] = 0
    
    for market in market_list:
        mult_i = specs.loc[market, "Multiplier"]
        df[market+' Price'] = df[market+'BClose']
        
        df[market+' Returns'] = (df[market+'BClose'] - (df[market+'BClose'].shift(1))) / (df[market+'SClose'].shift(1))

        df[market+' Vol'] = df[market+' Returns'].rolling(lookback).std()
        df[market+' Pct Change'] = (df[market+'BClose'] - (df[market+'BClose'].shift(lookback))) / (df[market+'SClose'].shift(lookback))
        df[market+' Trend'] = df[market+' Pct Change'] / df[market+' Vol']
        
        df[market+' P&L'] = 0
        df[market+' Position'] = 0
        
        df[market+' Pullback Sign'] = (np.sign(df[market+' Returns']) != np.sign(df[market+' Trend'])).shift(1)
        df[market+' Pullback Thld'] = (np.abs(df[market+' Returns']) >= np.abs(num_stdevs * df[market+' Vol'])).shift(1)
    
        # Append Transaction Cost to Dictionary 
        tick_i = specs.loc[market, 'Tick Size']
        exch_i = specs.loc[market, 'Exchange Fees']
        comm_i = specs.loc[market, 'Commission']
        nfa_i = specs.loc[market, 'NFA Fees']
        
        transac_cost[market] = (tick_i * mult_i) + exch_i + comm_i + nfa_i
        
        df['Sizing Sum'] += (1/(df[market+' Vol'] * mult_i))
    
    for market in market_list:
        pos_sign = np.sign(df[market+' Trend'])
        df[market+' Theo Position Size'] = pos_sign * 10 * np.abs((1/(df[market+' Vol'] * specs.loc[market, "Multiplier"])) / df['Sizing Sum'])
#         print(df[market+' Theo Position Size'])
    
#     print(transac_cost)
    df['Current Position'] = np.nan
    
    # Strongest trend
    df['Strongest Trend'] = (df.loc[:,['AD Trend', 'BP Trend', 'CD Trend', 'EC Trend', 'JY Trend', 'SF Trend']].abs().idxmax(axis=1)).str[:2]
    
    # Set initial position
    df.loc[start, 'Current Position'] = df.loc[start, 'Strongest Trend']
    df.loc[start, 'Take New Position'] = True
    
    # Pullback 
    conditions = [df['Strongest Trend'] == 'JY',
                  df['Strongest Trend'] == 'AD',
                  df['Strongest Trend'] == 'BP', 
                  df['Strongest Trend'] == 'CD',
                  df['Strongest Trend'] == 'EC',
                  df['Strongest Trend'] == 'SF']
#     values = [(df['JY Pullback Sign']),
#               df['AD Pullback Sign'],
#               df['BP Pullback Sign'],
#               df['CD Pullback Sign'],
#               df['EC Pullback Sign'],
#               (df['SF Pullback Sign'])]
    values = [(df['JY Pullback Sign'] & df['JY Pullback Thld']),
              df['AD Pullback Sign'] & df['AD Pullback Thld'],
              df['BP Pullback Sign'] & df['BP Pullback Thld'],
              df['CD Pullback Sign'] & df['CD Pullback Thld'],
              df['EC Pullback Sign'] & df['EC Pullback Thld'],
              (df['SF Pullback Sign'] & df['SF Pullback Thld'])]
                  
#     print(values[-1])
    df['Pullback'] = np.select(conditions, values, default="Invalid!")
#     df['Take New Position'] = np.where((df['Strongest Trend'] != df['Current Position']) & (df['Pullback']), True, False)
    df['Take New Position'] = np.where(df['Pullback'] == str(True), True, False)
#     print(np.any(df['Take New Position']))

    # Fill New Positions
    df['Current Position'] = np.where(df['Take New Position'], df['Strongest Trend'], df['Current Position'])
    
    # Fill holding positions 
    temp = df['Take New Position'].eq(1).cumsum()
#     print(df['Take New Position'].eq(1))
#     print(temp)
    df['Current Position'] = df.groupby(temp)['Current Position'].ffill()
#     print(df.loc[:, ['Current Position', 'Take New Position']].groupby(temp).ffill().head(30))
    
    df['Take New Position'] = np.where((df['Pullback'] == str(True)) & (df['Strongest Trend'] != (df['Current Position'].shift(1))), True, False)
    df.loc[start, 'Take New Position'] = True
                                       
    # Generate Position Size Held
    df['Position Size Held'] = np.nan
    pos_con = [df['Current Position'] == 'JY',
                  df['Current Position'] == 'AD',
                  df['Current Position'] == 'BP', 
                  df['Current Position'] == 'CD',
                  df['Current Position'] == 'EC',
                  df['Current Position'] == 'SF']
    pos_val = [(df['JY Theo Position Size']),
              df['AD Theo Position Size'],
              df['BP Theo Position Size'],
              df['CD Theo Position Size'],
              df['EC Theo Position Size'],
              (df['SF Theo Position Size'])]
    df['Position Size Held'] = np.where(df['Take New Position'] == True, np.select(pos_con, pos_val, default=np.nan), np.nan)
    
    temp2 = df['Current Position'].ne(df['Current Position'].shift(1)).cumsum()
#     print(temp2.iloc[15:35])
    df['Position Size Held'] = df.groupby(temp2)['Position Size Held'].ffill()
#     print(df.groupby(temp2)['Position Size Held'].head())
#     print(df['Position Size Held'])
    
    # Account for Transaction Costs (Multiplied by 2 since we have to both buy and sell the position)
    transac_con = [df['Take New Position'] & (df['Strongest Trend'] == 'JY'),
                  df['Take New Position'] & (df['Strongest Trend'] == 'AD'),
                  df['Take New Position'] & (df['Strongest Trend'] == 'BP'), 
                  df['Take New Position'] & (df['Strongest Trend'] == 'CD'),
                  df['Take New Position'] & (df['Strongest Trend'] == 'EC'),
                  df['Take New Position'] & (df['Strongest Trend'] == 'SF')]
    transac_values = [-2 * transac_cost['JY'] * df['JY Theo Position Size'],
                      -2 * transac_cost['AD'] * df['AD Theo Position Size'],
                      -2 * transac_cost['BP'] * df['BP Theo Position Size'],
                      -2 * transac_cost['CD'] * df['CD Theo Position Size'],
                      -2 * transac_cost['EC'] * df['EC Theo Position Size'],
                      -2 * transac_cost['SF'] * df['SF Theo Position Size']]
    df['Transac Cost'] = np.select(transac_con, transac_values, default=0)

    # Calculate P&L
    
    df['Total P&L'] = 0
    for market in market_list:
        mult_i = specs.loc[market, "Multiplier"]
        df[market+" Position One-Hot"] = np.where(df['Current Position'] == market, 1, 0)
        tmrw_bclose = df[market+"BClose"].shift(-1) 
        tdy_bclose = df[market+"BClose"]
        df[market+" P&L"] = (tmrw_bclose - tdy_bclose) * mult_i * df[market+" Position One-Hot"] * df['Position Size Held']
        df[market+" P&L"] = df[market+" P&L"].fillna(0)
        df[market+" P&L"] += df['Transac Cost'] * df[market+" Position One-Hot"]
#         print("P AND L")
#         print(df[market+" P&L"].cumsum())
        df['Total P&L'] += df[market+" P&L"].cumsum()
    
    return_df = df.copy()
    # Trim extraneous columns 
    for market in market_list:
        return_df.drop(columns=[market+'BClose',market+'SClose',
                                market+' Position One-Hot'], inplace=True)
    return return_df
    
    
    

In [5]:
vec = trade_logic(total)
vec.head()

,Sizing Sum,AD Price,AD Returns,AD Vol,AD Pct Change,AD Trend,AD P&L,AD Position,AD Pullback Sign,AD Pullback Thld,...,EC Theo Position Size,JY Theo Position Size,SF Theo Position Size,Current Position,Strongest Trend,Take New Position,Pullback,Position Size Held,Transac Cost,Total P&L
Date,,,,,,,,,,,,,,,,,,,,,
20051229,NaN,0.47275,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,Invalid!,NaN,0.0,0.0
20051230,NaN,0.47365,0.001230,NaN,NaN,NaN,0.0,0,True,False,...,NaN,NaN,NaN,NaN,NaN,False,Invalid!,NaN,0.0,0.0
20060103,NaN,0.48025,0.009010,NaN,NaN,NaN,0.0,0,True,False,...,NaN,NaN,NaN,NaN,NaN,False,Invalid!,NaN,0.0,0.0
20060104,NaN,0.48745,0.009742,NaN,NaN,NaN,0.0,0,True,False,...,NaN,NaN,NaN,NaN,NaN,False,Invalid!,NaN,0.0,0.0
20060105,NaN,0.48785,0.000536,NaN,NaN,NaN,0.0,0,True,False,...,NaN,NaN,NaN,NaN,NaN,False,Invalid!,NaN,0.0,0.0


In [398]:
vec.iloc[250:260,-20: ]

,SF Theo Position Size,Current Position,Strongest Trend,Pullback,Take New Position,Position Size Held,Transac Cost,Total P&L,AD Position One-Hot,BP Position One-Hot,CD Position One-Hot,EC Position One-Hot,JY Position One-Hot,SF Position One-Hot,AD Price,BP Price,CD Price,EC Price,JY Price,SF Price
Date,,,,,,,,,,,,,,,,,,,,
20061228,NaN,NaN,NaN,Invalid!,False,NaN,0.000000,0.000000,0,0,0,0,0,0,0.53425,1.9545,0.84495,1.4420,1.0250,1.0348
20061229,NaN,NaN,NaN,Invalid!,False,NaN,0.000000,0.000000,0,0,0,0,0,0,0.53335,1.9484,0.84155,1.4463,1.0239,1.0362
20070102,1.223496,BP,BP,False,True,2.718917,-47.417907,-3972.853914,0,1,0,0,0,0,0.54135,1.9654,0.84085,1.4560,1.0255,1.0414
20070103,1.220699,BP,BP,False,False,2.718917,0.000000,-5043.427370,0,1,0,0,0,0,0.53745,1.9423,0.83655,1.4438,1.0218,1.0322
20070104,1.233627,BP,BP,True,False,2.718917,0.000000,-7422.479495,0,1,0,0,0,0,0.52975,1.9360,0.83245,1.4358,1.0234,1.0272
20070105,-1.240823,BP,BP,True,False,2.718917,0.000000,-6164.980515,0,1,0,0,0,0,0.52385,1.9220,0.83565,1.4279,1.0266,1.0253
20070108,-1.237061,BP,BP,True,False,2.718917,0.000000,-5825.115926,0,1,0,0,0,0,0.52505,1.9294,0.83385,1.4288,1.0256,1.0246
20070109,-1.232558,BP,BP,False,False,2.718917,0.000000,-6946.669070,0,1,0,0,0,0,0.52665,1.9314,0.83345,1.4269,1.0211,1.0212
20070110,-1.233870,BP,BP,False,False,2.718917,0.000000,-5077.413829,0,1,0,0,0,0,0.52215,1.9248,0.83345,1.4205,1.0192,1.0181


In [404]:
total = df_list[0].copy()
for i in range(1, len(df_list)):
    total = pd.merge(total, df_list[i], on='Date', how='outer', sort=True).fillna(method='ffill')

In [201]:
# np.any((vec['Pullback'] != vec['Pullback2']))

In [390]:
vec.loc[20061221:20070105,['Current Position', 'Strongest Trend', 'Take New Position', 'Pullback', 'Position Size Held', 'Transac Cost', 'BP Theo Position Size', 'BP P&L', 'BPBClose']]

,Current Position,Strongest Trend,Take New Position,Pullback,Position Size Held,Transac Cost,BP Theo Position Size,BP P&L,BPBClose
Date,,,,,,,,,
20061221,NaN,NaN,False,Invalid!,NaN,0.000000,NaN,0.000000,1.9538
20061222,NaN,NaN,False,Invalid!,NaN,0.000000,NaN,0.000000,1.9482
20061226,NaN,NaN,False,Invalid!,NaN,0.000000,NaN,0.000000,1.9457
20061227,NaN,NaN,False,Invalid!,NaN,0.000000,NaN,0.000000,1.9481
20061228,NaN,NaN,False,Invalid!,NaN,0.000000,NaN,0.000000,1.9545
20061229,NaN,NaN,False,Invalid!,NaN,0.000000,NaN,0.000000,1.9484
20070102,BP,BP,True,False,2.718917,-47.417907,2.718917,-3972.853914,1.9654
20070103,BP,BP,False,False,2.718917,0.000000,2.704438,-1070.573456,1.9423
20070104,BP,BP,False,True,2.718917,0.000000,2.716202,-2379.052125,1.9360


In [391]:
vec.loc[20070102:20070129,:]

,ADBClose,ADSClose,BPBClose,BPSClose,CDBClose,CDSClose,ECBClose,ECSClose,JYBClose,JYSClose,SFBClose,SFSClose,Sizing Sum,AD Returns,AD Vol,AD Pct Change,AD Trend,AD P&L,AD Position,AD Pullback Sign,AD Pullback Thld,BP Returns,BP Vol,BP Pct Change,BP Trend,BP P&L,BP Position,BP Pullback Sign,BP Pullback Thld,CD Returns,CD Vol,CD Pct Change,CD Trend,CD P&L,CD Position,CD Pullback Sign,CD Pullback Thld,EC Returns,EC Vol,EC Pct Change,EC Trend,EC P&L,EC Position,EC Pullback Sign,EC Pullback Thld,JY Returns,JY Vol,JY Pct Change,JY Trend,JY P&L,JY Position,JY Pullback Sign,JY Pullback Thld,SF Returns,SF Vol,SF Pct Change,SF Trend,SF P&L,SF Position,SF Pullback Sign,SF Pullback Thld,AD Theo Position Size,BP Theo Position Size,CD Theo Position Size,EC Theo Position Size,JY Theo Position Size,SF Theo Position Size,Current Position,Strongest Trend,Pullback,Take New Position,Position Size Held,Transac Cost,Total P&L,AD Position One-Hot,BP Position One-Hot,CD Position One-Hot,EC Position One-Hot,JY Position One-Hot,SF Position One-Hot
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20070102,0.54135,0.7946,1.9654,1.9742,0.84085,0.8595,1.4560,1.3333,1.0255,0.8500,1.0414,0.8306,0.011987,0.010170,0.005543,0.093767,16.916990,-0.0,0,True,False,0.008686,0.004909,0.140951,28.710671,-3972.853914,0,True,False,-0.000814,0.004544,-0.012087,-2.660272,-0.0,0,True,False,0.007328,0.004687,0.098898,21.100657,-0.0,0,True,False,0.001886,0.005164,-0.057477,-11.130331,-0.000000,0,True,False,0.006300,0.005455,0.045152,8.277336,-0.0,0,True,False,1.505123,2.718917,-1.836083,1.423955,-1.292427,1.223496,BP,BP,False,True,2.718917,-47.417907,-3972.853914,0,1,0,0,0,0
20070103,0.53745,0.7907,1.9423,1.9511,0.83655,0.8552,1.4438,1.3211,1.0218,0.8463,1.0322,0.8214,0.011915,-0.004908,0.005552,0.087099,15.686525,0.0,0,False,True,-0.011701,0.004965,0.130564,26.294361,-1070.573456,0,False,True,-0.005003,0.004553,-0.018910,-4.153366,0.0,0,False,True,-0.009150,0.004725,0.089646,18.974396,0.0,0,False,True,-0.004353,0.005170,-0.060812,-11.761753,0.000000,0,True,True,-0.011076,0.005500,0.034091,6.197836,0.0,0,False,True,1.511579,2.704438,-1.843474,1.421157,-1.298652,1.220699,BP,BP,False,False,2.718917,0.000000,-5043.427370,0,1,0,0,0,0
20070104,0.52975,0.7830,1.9360,1.9448,0.83245,0.8511,1.4358,1.3131,1.0234,0.8479,1.0272,0.8164,0.012064,-0.009738,0.005562,0.066973,12.042222,0.0,0,True,True,-0.003229,0.004883,0.109978,22.524519,-2379.052125,0,True,True,-0.004794,0.004549,-0.029072,-6.391602,0.0,0,False,True,-0.006056,0.004640,0.066131,14.250898,0.0,0,True,True,0.001891,0.005067,-0.073887,-14.580477,0.000000,0,False,True,-0.006087,0.005375,0.007815,1.453799,0.0,0,True,True,1.490375,2.716202,-1.822296,1.428958,-1.308541,1.233627,BP,BP,True,False,2.718917,0.000000,-7422.479495,0,1,0,0,0,0
20070105,0.52385,0.7771,1.9220,1.9308,0.83565,0.8543,1.4279,1.3052,1.0266,0.8511,1.0253,0.8145,0.012091,-0.007535,0.005551,0.048774,8.786764,0.0,0,True,True,-0.007199,0.004886,0.093805,19.199528,1257.498980,0,True,True,0.003760,0.004527,-0.032967,-7.281793,0.0,0,False,True,-0.006016,0.004627,0.050530,10.920608,0.0,0,True,True,0.003774,0.005074,-0.070632,-13.921033,0.000000,0,True,True,-0.002327,0.005332,-0.005575,-1.045570,0.0,0,True,True,1.489956,2.708422,-1.826799,1.429948,-1.304052,-1.240823,BP,BP,True,False,2.718917,0.000000,-6164.980515,0,1,0,0,0,0
20070108,0.52505,0.7783,1.9294,1.9382,0.83385,0.8525,1.4288,1.3061,1.0256,0.8501,1.0246,0.8138,0.012127,0.001544,0.005551,0.049819,8.974095,0.0,0,True,True,0.003833,0.004890,0.099203,20.287667,339.864589,0,True,True,-0.002107,0.004449,-0.021815,-4.902958,0.0,0,True,True,0.000690,0.004626,0.052826,11.419714,0.0,0,True,True,-0.001175,0.005074,-0.072209,-14.231598,0.000000,0,True,True,-0.000859,0.005333,-0.006336,-1.188239,0.0,0,False,True,1.485351,2.698136,-1.853285,1.426031,-1.300136,-1.237061,BP,BP,True,False,2.718917,0.000000,-5825.115926,0,1,0,0,0,0
20070109,0.52665,0.7799,1.9314,1.9402,0.83345,0.8521,1.4269,1.30

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [392]:
vec.tail(105)

,ADBClose,ADSClose,BPBClose,BPSClose,CDBClose,CDSClose,ECBClose,ECSClose,JYBClose,JYSClose,SFBClose,SFSClose,Sizing Sum,AD Returns,AD Vol,AD Pct Change,AD Trend,AD P&L,AD Position,AD Pullback Sign,AD Pullback Thld,BP Returns,BP Vol,BP Pct Change,BP Trend,BP P&L,BP Position,BP Pullback Sign,BP Pullback Thld,CD Returns,CD Vol,CD Pct Change,CD Trend,CD P&L,CD Position,CD Pullback Sign,CD Pullback Thld,EC Returns,EC Vol,EC Pct Change,EC Trend,EC P&L,EC Position,EC Pullback Sign,EC Pullback Thld,JY Returns,JY Vol,JY Pct Change,JY Trend,JY P&L,JY Position,JY Pullback Sign,JY Pullback Thld,SF Returns,SF Vol,SF Pct Change,SF Trend,SF P&L,SF Position,SF Pullback Sign,SF Pullback Thld,AD Theo Position Size,BP Theo Position Size,CD Theo Position Size,EC Theo Position Size,JY Theo Position Size,SF Theo Position Size,Current Position,Strongest Trend,Pullback,Take New Position,Position Size Held,Transac Cost,Total P&L,AD Position One-Hot,BP Position One-Hot,CD Position One-Hot,EC Position One-Hot,JY Position One-Hot,SF Position One-Hot
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20210922,0.72455,0.72415,1.3620,1.3616,0.78265,0.78250,1.17375,1.17115,0.91160,0.91070,1.0841,1.0811,0.014222,0.000484,0.005785,0.007673,1.326381,0.0,0,True,True,-0.003586,0.004503,0.067117,14.905158,1840.543300,0,False,True,0.002627,0.004165,0.040583,9.743967,0.0,0,False,True,-0.002852,0.003532,-0.009679,-2.740152,0.0,0,True,True,-0.005949,0.003425,-0.049578,-14.474367,0.000000,0,True,True,-0.003594,0.004126,-0.018264,-4.426820,0.0,0,True,True,1.215439,2.498361,1.688198,-1.592385,-1.642211,-1.363406,BP,BP,False,False,2.495652,0.000000,79257.461297,0,1,0,0,0,0
20210923,0.73155,0.73115,1.3738,1.3734,0.79060,0.79045,1.17910,1.17650,0.90850,0.90760,1.0879,1.0849,0.014192,0.009667,0.005761,0.030376,5.272344,0.0,0,False,True,0.008666,0.004532,0.077987,17.206366,-1138.641194,0,True,True,0.010160,0.004198,0.056715,13.509852,0.0,0,False,True,0.004568,0.003535,-0.000985,-0.278611,0.0,0,False,True,-0.003404,0.003419,-0.048339,-14.136910,0.000000,0,False,True,0.003515,0.004121,-0.010143,-2.461023,0.0,0,False,True,1.223028,2.487397,1.678474,-1.594783,-1.648580,-1.367738,BP,BP,True,False,2.495652,0.000000,78118.820103,0,1,0,0,0,0
20210924,0.72630,0.72590,1.3665,1.3661,0.78985,0.78970,1.17595,1.17335,0.90395,0.90305,1.0860,1.0830,0.014168,-0.007180,0.005775,0.026730,4.628728,0.0,0,False,True,-0.005315,0.004545,0.070061,15.415216,764.293404,0,False,True,-0.000949,0.004197,0.053598,12.769940,0.0,0,False,True,-0.002677,0.003538,-0.004364,-1.233423,0.0,0,True,True,-0.005013,0.003433,-0.053031,-15.448281,0.000000,0,False,True,-0.001751,0.004120,-0.009429,-2.288564,0.0,0,True,True,1.222255,2.484798,1.681676,-1.595800,-1.644909,-1.370563,BP,JY,False,False,2.495652,0.000000,78883.113507,0,1,0,0,0,0
20210927,0.72925,0.72885,1.3714,1.3710,0.79240,0.79225,1.17430,1.17170,0.90215,0.90125,1.0850,1.0820,0.014171,0.004064,0.005775,0.034728,6.013448,0.0,0,True,True,0.003587,0.004549,0.074833,16.450005,-2745.217125,0,True,True,0.003229,0.004196,0.060470,14.412629,0.0,0,True,True,-0.001406,0.003532,-0.002019,-0.571598,0.0,0,False,True,-0.001993,0.003433,-0.053281,-15.519059,0.000000,0,False,True,-0.000923,0.004117,-0.007785,-1.890870,0.0,0,False,True,1.221944,2.481999,1.681946,-1.598570,-1.644334,-1.371208,BP,BP,True,False,2.495652,0.000000,76137.896382,0,1,0,0,0,0
20210928,0.72475,0.72435,1.3538,1.3534,0.78880,0.78865,1.17205,1.16945,0.89745,0.89655,1.0802,1.0772,0.014123,-0.006174,0.005776,0.021924,3.795773,0.0,0,False,True,-0.012837,0.004597,0.052418,11.403913,-1544.184633,0,False,True,-0.004544,0.004205,0.053673,12.763600,0.0,0,False,True,-0.001920,0.003526,-0.007490,-2.123838,0.0,0,False,True,-0.005215,0.003447,-0.059034,-17.126040,0.000000,0,False,True,-0.004436,0.004117,-0.016608,-4.034300,0.0,0,False,True,1.225918,2.464697,1.683805,-1.606317,-1.643295,-1.375968,BP,JY,False,False,2.495652,0.000000,74593.711749,0,1,0,0,0,0
20210929,0.71900,0.71860,1.3

In [6]:
vec.tail()

,Sizing Sum,AD Price,AD Returns,AD Vol,AD Pct Change,AD Trend,AD P&L,AD Position,AD Pullback Sign,AD Pullback Thld,...,EC Theo Position Size,JY Theo Position Size,SF Theo Position Size,Current Position,Strongest Trend,Take New Position,Pullback,Position Size Held,Transac Cost,Total P&L
Date,,,,,,,,,,,,,,,,,,,,,
20220214,0.013999,0.71225,-0.000561,0.005892,-0.084316,-14.311086,0.0,0,False,True,...,-1.500838,-1.567838,-1.328457,JY,JY,False,True,-1.637288,0.0,-5790.078846
20220215,0.014017,0.71530,0.004282,0.005898,-0.078903,-13.377453,0.0,0,False,False,...,-1.499763,-1.565761,-1.336066,JY,JY,False,False,-1.637288,0.0,-6138.002450
20220216,0.014029,0.72030,0.006990,0.005914,-0.074660,-12.624978,0.0,0,True,True,...,-1.500583,-1.564520,-1.332532,JY,JY,False,False,-1.637288,0.0,-6946.413177
20220217,0.014047,0.71935,-0.001319,0.005858,-0.087338,-14.909149,0.0,0,True,True,...,-1.499358,-1.558183,-1.330712,JY,JY,False,True,-1.637288,0.0,-6629.188714
20220218,0.014071,0.71865,-0.000973,0.005837,-0.094959,-16.267246,0.0,0,False,False,...,-1.499763,-1.560474,-1.328614,JY,JY,False,True,-1.637288,0.0,-6629.188714


In [407]:
vec.index[-1]

20220218